In [1]:
import datetime
#import Image
import gc
import numpy as np
import os
import random
from scipy import misc
import string
import time
import sys
import sklearn.metrics as skm

# Set some Theano config before initializing
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cpu,floatX=float32,allow_gc=False,openmp=True"
#import theano

# MatPlotLib - Setup for Jupyter notebook output
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

# Our modules
import dwdii_bc_model_helper as bc
#import bc_models as models

# And Keras so we can emit the version
#import keras

random.seed(20275)
np.set_printoptions(precision=2)

In [5]:
imagePath = "ddsm_png"
#imagePath = "/root/bc_data/Data_Thresholded/DDSM"
trainDataPath = "data/ddsm_train.csv"
testDataPath = "data/ddsm_test.csv"
imgResize = (75, 75)

In [6]:
os.listdir('data')

['ddsm_test.csv', 'ddsm_train.csv', 'ddsm_val.csv', 'mias_all.csv']

In [7]:
metaData, meta2, mCounts = bc.load_training_metadata(trainDataPath, balanceViaRemoval=True, verbose=True)

Raw Balance
----------------
benign 531
malignant 739
normal 2685
balanaceViaRemoval.avgE: 1318
balanaceViaRemoval.theshold: 1318.0

After Balancing
----------------
benign 531
malignant 739
normal 862


In [8]:
# Actually load some representative data for model experimentation
maxData = len(metaData)
X_data, Y_data = bc.load_data(trainDataPath, imagePath, maxData = maxData, verboseFreq = 50, imgResize=imgResize)
print X_data.shape
print Y_data.shape

Raw Balance
----------------
benign 531
malignant 739
normal 2685
balanaceViaRemoval.avgE: 1318
balanaceViaRemoval.theshold: 1318.0

After Balancing
----------------
benign 531
malignant 739
normal 862


dwdii_bc_model_helper.py:162: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y_data = np.zeros([total, 1], dtype=np.int8)


0.0000: A_0152_1.RIGHT_MLO.LJPEG.png
Not Found: ddsm_png\3\B_3158_1.LEFT_MLO.LJPEG.png
Not Found: ddsm_png\0\C_0270_1.RIGHT_CC.LJPEG.png
Not Found: ddsm_png\3\B_3422_1.RIGHT_CC.LJPEG.png
Not Found: ddsm_png\0\C_0283_1.LEFT_MLO.LJPEG.png
Not Found: ddsm_png\3\B_3125_1.LEFT_MLO.LJPEG.png
Not Found: ddsm_png\0\C_0381_1.RIGHT_MLO.LJPEG.png
Not Found: ddsm_png\3\B_3154_1.RIGHT_CC.LJPEG.png
Not Found: ddsm_png\0\C_0313_1.LEFT_MLO.LJPEG.png
Not Found: ddsm_png\0\C_0383_1.RIGHT_MLO.LJPEG.png
Not Found: ddsm_png\3\B_3104_1.LEFT_MLO.LJPEG.png
Not Found: ddsm_png\3\B_3119_1.RIGHT_CC.LJPEG.png
Not Found: ddsm_png\3\B_3149_1.RIGHT_MLO.LJPEG.png
Not Found: ddsm_png\0\C_0376_1.RIGHT_CC.LJPEG.png
Not Found: ddsm_png\0\C_0325_1.LEFT_MLO.LJPEG.png
Not Found: ddsm_png\3\B_3117_1.RIGHT_MLO.LJPEG.png
Not Found: ddsm_png\3\B_3422_1.RIGHT_MLO.LJPEG.png
Not Found: ddsm_png\0\C_0508_1.RIGHT_MLO.LJPEG.png
Not Found: ddsm_png\0\C_0310_1.LEFT_CC.LJPEG.png
Not Found: ddsm_png\3\B_3455_1.LEFT_MLO.LJPEG.png
0.0235: 

In [9]:
# Actually load some representative data for model experimentation
maxData = len(metaData)
X_test, Y_test = bc.load_data(testDataPath, imagePath, maxData = maxData, verboseFreq = 50, imgResize=imgResize)
print X_test.shape
print Y_test.shape

Raw Balance
----------------
benign 142
malignant 179
normal 658
balanaceViaRemoval.avgE: 326
balanaceViaRemoval.theshold: 326.0

After Balancing
----------------
benign 142
malignant 179
normal 215
0.0000: B_3380_1.RIGHT_MLO.LJPEG.png
Not Found: ddsm_png\3\B_3152_1.LEFT_MLO.LJPEG.png
Not Found: ddsm_png\3\B_3421_1.LEFT_MLO.LJPEG.png
Not Found: ddsm_png\3\B_3092_1.RIGHT_CC.LJPEG.png
Not Found: ddsm_png\3\B_3133_1.RIGHT_CC.LJPEG.png
Not Found: ddsm_png\0\C_0491_1.LEFT_MLO.LJPEG.png
Not Found: ddsm_png\0\C_0255_1.RIGHT_MLO.LJPEG.png
Not Found: ddsm_png\0\C_0240_1.RIGHT_CC.LJPEG.png
Not Found: ddsm_png\3\B_3160_1.RIGHT_CC.LJPEG.png
Not Found: ddsm_png\3\B_3114_1.LEFT_CC.LJPEG.png
Not Found: ddsm_png\0\C_0286_1.RIGHT_CC.LJPEG.png
Not Found: ddsm_png\0\C_0323_1.LEFT_CC.LJPEG.png
Not Found: ddsm_png\3\B_3096_1.LEFT_MLO.LJPEG.png
Not Found: ddsm_png\3\B_3366_1.RIGHT_MLO.LJPEG.png
Not Found: ddsm_png\0\C_0389_1.LEFT_CC.LJPEG.png
Not Found: ddsm_png\0\C_0320_1.LEFT_CC.LJPEG.png
Not Found: ddsm_

In [10]:
X_train = X_data
Y_train = Y_data

In [11]:
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape

(1508L, 75L, 75L)
(375L, 75L, 75L)
(1508L, 1L)
(375L, 1L)


In [12]:
import collections
def yDist(y):
    bcCounts = collections.defaultdict(int)
    for a in range(0, y.shape[0]):
        bcCounts[y[a][0]] += 1
    return bcCounts

print "Y_train Dist: " + str(yDist(Y_train))
print "Y_test Dist: " + str(yDist(Y_test))

Y_train Dist: defaultdict(<type 'int'>, {0: 719, 1: 50, 2: 739})
Y_test Dist: defaultdict(<type 'int'>, {0: 184, 1: 12, 2: 179})


In [13]:
# Load the bc array for our count in the model definition
bcTypes = bc.bcNumerics()
print bcTypes
print len(bcTypes)

{'benign': 1, 'malignant': 2, 'normal': 0}
3


In [14]:
X_train_s = X_train.reshape((1508,-1))

In [15]:
X_test_s = X_test.reshape((375,-1))

In [16]:
from sklearn.svm import SVC

In [17]:
model = SVC(gamma=0.001)

In [18]:
model.fit(X_train_s,Y_train)

C:\Anaconda2\lib\site-packages\sklearn\svm\base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [19]:
predicted = model.predict(X_test_s)
expected = Y_test

In [20]:
from sklearn import metrics

In [21]:
print metrics.accuracy_score(expected,predicted)

0.709333333333
